In [1]:
!pip install requests


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install opencv-python


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install tensorflow


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import requests
from io import BytesIO
from PIL import Image
from skimage import exposure, transform
import random
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
import string
import os
import re
import pickle
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import math
from skimage.transform import resize

In [2]:
df=pd.read_csv("A2_Data.csv")

In [58]:
df

,Unnamed: 0,Image,Review Text,Processed_Review,TF
0,3452,https://images-na.ssl-images-amazon.com/images...,Loving these vintage springs on my vintage str...,"[loving, vintage, springs, vintage, strat, goo...","{'loving': 0.06666666666666667, 'vintage': 0.1..."
1,1205,https://images-na.ssl-images-amazon.com/images...,Works great as a guitar bench mat. Not rugged ...,"[works, great, guitar, bench, mat, rugged, eno...","{'works': 0.041666666666666664, 'great': 0.041..."
2,1708,https://images-na.ssl-images-amazon.com/images...,We use these for everything from our acoustic ...,"[use, everything, acoustic, bass, ukuleles, kn...","{'use': 0.03278688524590164, 'everything': 0.0..."
3,2078,https://images-na.ssl-images-amazon.com/images...,Great price and good quality. It didn't quite...,"[great, price, good, quality, nt, quite, match...","{'great': 0.08333333333333333, 'price': 0.0833..."
4,801,https://images-na.ssl-images-amazon.com/images...,I bought this bass to split time as my primary...,"[bought, bass, split, time, primary, bass, dea...","{'bought': 0.02127659574468085, 'bass': 0.1063..."
...,...,...,...,...,...
995,1265,https://images-na.ssl-images-amazon.com/images...,Extremely impressed with this kit.,"[extremely, impressed, kit]","{'extremely': 0.3333333333333333, 'impressed':..."
996,1882,https://images-na.ssl-images-amazon.com/images...,This is a great stereo reverb with plenty of c...,"[great, stereo, reverb, plenty, control, get, ...","{'great': 0.13636363636363635, 'stereo': 0.045..."
997,1547,https://images-na.ssl-images-amazon.com/images...,I really like the simplicity of this bridge. I...,"[really, like, simplicity, bridge, adjusts, ea...","{'really': 0.018518518518518517, 'like': 0.018..."
998,1004,https://images-na.ssl-images-amazon.com/images...,"Great Product, but there is no warranty in the...","[great, product, warranty, usa, purchased, ama...","{'great': 0.08333333333333333, 'product': 0.08..."


In [4]:
def convert_to_lowercase(input_string):
    return input_string.lower()

def tokenize_words(input_text):
    return word_tokenize(input_text)

def filter_stopwords_from_tokens(word_list, stop_words):
    filtered_words = [word for word in word_list if word not in stop_words]
    return filtered_words

def remove_punctuation_from_tokens(tokens):
    punctuation_pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    tokens_without_punctuation = [punctuation_pattern.sub('', token) for token in tokens]
    return tokens_without_punctuation

def filter_non_blank_tokens(input_tokens):
    non_blank_tokens = [token for token in input_tokens if token.strip()]
    return non_blank_tokens

def apply_lemmatization(word_list, lemmatizer):
    lemmatized_words = [lemmatizer.lemmatize(word) for word in word_list]
    return lemmatized_words


In [6]:
def preprocess_text(input_text, stop_words,lemmatizer):
    if pd.isna(input_text):
        return []
    
    lowercase_text = convert_to_lowercase(input_text)
    word_tokens = tokenize_words(lowercase_text)
    tokens_without_stopwords = filter_stopwords_from_tokens(word_tokens, stop_words)
    tokens_without_punctuation = remove_punctuation_from_tokens(tokens_without_stopwords)
    non_blank_tokens = filter_non_blank_tokens(tokens_without_punctuation)
    lemmatized_tokens = apply_lemmatization(non_blank_tokens, lemmatizer)
    return non_blank_tokens
   

In [7]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
df['Processed_Review'] = df['Review Text'].apply(lambda x: preprocess_text(x, stop_words,lemmatizer))

In [8]:
df

,Unnamed: 0,Image,Review Text,Processed_Review
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...,"[loving, vintage, springs, vintage, strat, goo..."
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...,"[works, great, guitar, bench, mat, rugged, eno..."
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...,"[use, everything, acoustic, bass, ukuleles, kn..."
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...,"[great, price, good, quality, nt, quite, match..."
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...,"[bought, bass, split, time, primary, bass, dea..."
...,...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.,"[extremely, impressed, kit]"
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...,"[great, stereo, reverb, plenty, control, get, ..."
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...,"[really, like, simplicity, bridge, adjusts, ea..."
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the...","[great, product, warranty, usa, purchased, ama..."


In [9]:
def calculate_tf(tokens):
    tf_counter = Counter(tokens)  
    total_words = len(tokens)
    tf = {word: count / total_words for word, count in tf_counter.items()} 
    return tf

df['TF'] = df['Processed_Review'].apply(calculate_tf)

In [10]:
df['TF']

0      {'loving': 0.06666666666666667, 'vintage': 0.1...
1      {'works': 0.041666666666666664, 'great': 0.041...
2      {'use': 0.03278688524590164, 'everything': 0.0...
3      {'great': 0.08333333333333333, 'price': 0.0833...
4      {'bought': 0.02127659574468085, 'bass': 0.1063...
                             ...                        
995    {'extremely': 0.3333333333333333, 'impressed':...
996    {'great': 0.13636363636363635, 'stereo': 0.045...
997    {'really': 0.018518518518518517, 'like': 0.018...
998    {'great': 0.08333333333333333, 'product': 0.08...
999    {'product': 0.1, 'good': 0.1, 'used': 0.1, 'pr...
Name: TF, Length: 1000, dtype: object

In [11]:
def calculate_idf(documents):
    N = len(documents)
    all_words = set([word for doc in documents for word in doc])
    idf = {}
    for word in all_words:
        doc_with_word = sum([1 for doc in documents if word in doc])
        idf[word] = math.log(N / doc_with_word)
    return idf

all_reviews = df['Processed_Review'].tolist()
idf_scores = calculate_idf(all_reviews)

In [12]:
def calculate_tfidf(tf, idf):
    tfidf = {}
    for word, tf_score in tf.items():
        if word in idf:
            tfidf[word] = tf_score * idf[word]
        else:
            # If word not found in IDF scores, assign TF-IDF score of 0
            tfidf[word] = 0
    return tfidf
tfidf_scores = df['TF'].apply(lambda tf: calculate_tfidf(tf, idf_scores))

In [13]:
def calculate_tfidf_vector(tfidf_scores, vocabulary):
    tfidf_vector = np.zeros(len(vocabulary))
    for word, score in tfidf_scores.items():
        if word in vocabulary:
            index = vocabulary.index(word)
            tfidf_vector[index] = score
    return tfidf_vector

#  list of all unique words (vocabulary) across all documents
vocabulary = list(idf_scores.keys())

# Create TF-IDF vectors for each document
tfidf_vectors = []
for document_tfidf in tfidf_scores:
    tfidf_vector = calculate_tfidf_vector(document_tfidf, vocabulary)
    tfidf_vectors.append(tfidf_vector)

# Convert tfidf_vectors to numpy array for further analysis
tfidf_vectors = np.array(tfidf_vectors)

# pickle file saving for TF-IDF

In [14]:
with open('tfidf_vectors.pkl', 'wb') as f:
    pickle.dump(tfidf_vectors, f)


# images

In [18]:
df['Image'] = df['Image'].str.replace('[\[\]\'\']', '', regex=True)


In [19]:
# Initialize VGG16 model, pre-trained on ImageNet
base_model = VGG16(weights='imagenet')
feature_model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)  # Using 'fc1' layer for feature extraction

# Image preprocessing function with enhancements
def preprocess_image(img_urls):
    
    img_urls = img_urls.strip('][').split(', ')

    for img_url in img_urls:
        img_url = img_url.strip('\'"')  

        try:
            # Download the image from the URL
            response = requests.get(img_url)
            img = Image.open(BytesIO(response.content))
            img = img.convert('RGB')  # Ensure image is RGB

            # Resize and convert the image
            img = img.resize((224, 224))  
            img = np.array(img) 

            # Apply random transformations (as before)
            img = np.rot90(img, k=1)
            # Adjust brightness (as before)
            brightness_factor = 0.8
            img = img * brightness_factor  # Adjust brightness
            img = np.clip(img, 0, 255)     # Ensure values are in proper range

            # Preprocess for model input
            img = np.expand_dims(img, axis=0)  # Add batch dimension
            img = preprocess_input(img)       # Preprocess according to VGG16

            return img
        except Exception as e:
            print(f"An error occurred with {img_url}: {e}")
            continue  # Move to the next URL if any issues arise

    return None

# Feature extraction function
def extract_features(model, preprocessed_img):
    """
    Extract features from preprocessed image using VGG16
    """
    if preprocessed_img is not None:  # Check if the image was successfully preprocessed
        features = model.predict(preprocessed_img)
        return features.flatten()  # Flatten the features to a 1D array
    else:
        return None  # Return None if there was an issue with preprocessing

# Preprocess and extract features for all images
all_features = []
for img_urls in df['Image']:
    preprocessed_img = preprocess_image(img_urls)
    features = extract_features(feature_model, preprocessed_img)
    if features is not None:
        all_features.append(features)
    else:
        all_features.append([None] * 4096)  # Replace with the correct dimensionality for your chosen layer

# Convert the list of features to a DataFrame
features_df = pd.DataFrame(all_features)




1/1 [==============================] - 0s 262ms/step
An error occurred with https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x00000216EE14EDE0>
An error occurred with https://images-na.ssl-images-amazon.com/images/I/71wHUWncMGL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x00000216EE427790>
1/1 [==============================] - 0s 197ms/step
An error occurred with https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x00000216EE166890>
An error occurred with https://images-na.ssl-images-amazon.com/images/I/81SX3oAWbNL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x00000216EE1AD440>
1/1 [==============================] - 0s 199ms/step
An error occurred with https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg: cannot identify image file <_io.BytesIO object at 0x00000216EE49FC90>
1/1 [=========

In [20]:
scaler = StandardScaler()
all_features = scaler.fit_transform(all_features)

In [21]:
valid_indices = ~np.any(np.isnan(all_features), axis=1)
valid_features = all_features[valid_indices]
valid_data = df[valid_indices]

# saving pickle file Extracted Features

In [22]:
with open('valid_features.pkl', 'wb') as f:
    pickle.dump(valid_features, f)


In [28]:
def find_similar_images(input_features, valid_features, top_n=3):
    """
    Find similar images based on cosine similarity between input features and valid features.
    """
    similarity_scores = cosine_similarity([input_features], valid_features)[0]
    similar_indices = similarity_scores.argsort()[-top_n:][::-1]
    return similar_indices, similarity_scores[similar_indices]

In [69]:
def compute_cosine_similarity(vector1, vector2):
    """
    Compute cosine similarity between two vectors or matrices of vectors.

    Args:
    - vector1: First vector or matrix of vectors with shape (n, d).
    - vector2: Second vector or matrix of vectors with shape (m, d).

    Returns:
    - Similarity: Cosine similarity between the two vectors or matrices with shape (n, m).
    """
    dot_product = np.dot(vector1, vector2.T)
    norm1 = np.linalg.norm(vector1, axis=1, keepdims=True)
    norm2 = np.linalg.norm(vector2, axis=1, keepdims=True)
    
    # Replace NaN values with zeros
    dot_product = np.nan_to_num(dot_product)
    norm1 = np.nan_to_num(norm1)
    norm2 = np.nan_to_num(norm2)
    
    # Handle division by zero
    norm_product = norm1 * norm2.T
    norm_product[norm_product == 0] = 1  # Avoid division by zero
    
    similarity = dot_product / norm_product
    return similarity
    



In [70]:
def find_similar_images(input_features, valid_features, top_n=3):
    """
    Find similar images based on cosine similarity between input features and valid features.
    """
    similarity_scores = compute_cosine_similarity([input_features], valid_features)[0]
    similar_indices = similarity_scores.argsort()[-top_n:][::-1]
    return similar_indices, similarity_scores[similar_indices]

In [72]:
# Load TF-IDF scores from pickle file
with open('tfidf_vectors.pkl', 'rb') as f:
    tfidf_vectors = pickle.load(f)

# Load valid features from pickle file
with open('valid_features.pkl', 'rb') as f:
    valid_features = pickle.load(f)

# Take user input for text and image URL
input_text = input("Enter the text: ")
input_image_url = input("Enter the image URL: ")

# Tokenize and preprocess the input text
input_tokens = preprocess_text(input_text, stop_words, lemmatizer)  # Assuming you have defined preprocess_text function
input_tf = calculate_tf(input_tokens)
input_tfidf = calculate_tfidf(input_tf, idf_scores)
input_tfidf_vector = calculate_tfidf_vector(input_tfidf, vocabulary)

# Preprocess and extract features for the input image
input_image = preprocess_image(input_image_url)  # Assuming you have defined preprocess_image function
input_features = extract_features(feature_model, input_image)  # Assuming you have defined extract_features function

# Calculate cosine similarity between input text and reviews
text_similarities = compute_cosine_similarity(input_tfidf_vector.reshape(1, -1), tfidf_vectors)

# Calculate cosine similarity between input image and valid images
if input_features is not None:
    input_features = scaler.transform([input_features])  # Assuming you have defined scaler
    image_similarities = compute_cosine_similarity(input_features, valid_features)
else:
    image_similarities = np.zeros(len(valid_features))  # Set similarities to zero if image processing failed

# Get indices of top 3 most similar texts based on text
top_text_indices = text_similarities.argsort()[0][-3:][::-1]

# Filter out text indices with different Review ID
unique_text_indices = []
review_ids = set()
for text_index in top_text_indices:
    review_id = df.iloc[text_index]['Unnamed: 0']
    if review_id not in review_ids:
        unique_text_indices.append(text_index)
        review_ids.add(review_id)

# Get indices of top 3 most similar images based on image
top_image_indices = image_similarities.argsort()[0][-3:][::-1]
print("\nTop 3 similar texts and their corresponding image URLs:")
text_image_similarity_scores = []
for idx, text_index in enumerate(top_text_indices):
    review_id = df.iloc[text_index]['Unnamed: 0']
    text_similarity_score = text_similarities[0][text_index]
    review_text = df.iloc[text_index]['Review Text']  # Get the review text
    
    # Print corresponding image URL for the text
    image_urls = valid_data[valid_data['Unnamed: 0'] == review_id]['Image'].tolist()
    print(f"{idx + 1}) Review ID: {review_id}")
    print(f"   Image URLs: {image_urls}")
    print(f"   Review: {review_text}")
    print(f"   Cosine similarity of text: {text_similarity_score}")

    # Calculate cosine similarity between corresponding image URL and entered image URL
    entered_image_features = preprocess_image(input_image_url)
    entered_image_features = extract_features(feature_model, entered_image_features)
    entered_image_features = scaler.transform([entered_image_features])
    image_similarity_scores = []
    for image_url in image_urls:
        corresponding_image_features = preprocess_image(image_url)
        corresponding_image_features = extract_features(feature_model, corresponding_image_features)
        corresponding_image_features = scaler.transform([corresponding_image_features])
        image_similarity_score = compute_cosine_similarity(entered_image_features, corresponding_image_features)[0][0]
        image_similarity_scores.append(image_similarity_score)
        print(f"      Cosine similarity of image: {image_similarity_score}")

    avg_image_similarity = np.mean(image_similarity_scores)
    composite_similarity = (text_similarity_score + avg_image_similarity) / 2
    print(f"   Composite similarity score: {composite_similarity}")
    print()

print("\nTop 3 similar images and their corresponding text review IDs:")
for idx, image_index in enumerate(top_image_indices):
    review_id = valid_data.iloc[image_index]['Unnamed: 0']
    image_similarity_score = image_similarities[0][image_index]
    
    # Print corresponding text review ID for the image
    text_review_ids = df[df['Unnamed: 0'] == review_id]['Unnamed: 0'].tolist()
    print(f"{idx + 1}) Image URL: {valid_data.iloc[image_index]['Image']}")
    print(f"Review IDs: {text_review_ids}")
    
    # Calculate cosine similarity between image and text
    text_similarity_scores = []
    for text_review_id in text_review_ids:
        text_index = df[df['Unnamed: 0'] == text_review_id].index[0]
        text_similarity_score = text_similarities[0][text_index]
        text_review_text = df.iloc[text_index]['Review Text']
        print(f"   Review Text: {text_review_text}")
        print(f"   Cosine similarity of text: {text_similarity_score}")

        text_similarity_scores.append(text_similarity_score)
    
    avg_text_similarity = np.mean(text_similarity_scores)
    
    # Calculate cosine similarity between input image and top similar image
    input_image_features = preprocess_image(input_image_url)
    input_image_features = extract_features(feature_model, input_image_features)
    input_image_features = scaler.transform([input_image_features])
    corresponding_image_features = valid_features[image_index].reshape(1, -1)
    image_similarity = compute_cosine_similarity(input_image_features, corresponding_image_features)[0][0]
    print(f"   Cosine similarity of image for rank {idx + 1}: {image_similarity}")
    
    composite_similarity = (image_similarity_score + avg_text_similarity) / 2
    print(f"Composite similarity score: {composite_similarity}")
    print()


Enter the text: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg
1/1 [==============================] - 0s 102ms/step

Top 3 similar texts and their corresponding image URLs:
1) Review ID: 654
   Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg']
   Review: I have been using Fender locking tuners for about five years on various strats and teles. Definitely helps with tuning stability and way faster to restring if there is a break.
   Cosine similarity of text: 1.0
1/1 [==============================] - 0s 104ms/step
      Cosine similarity of image: 1.0000000000000002
   Composite similarity score: 1.0

2) Review ID: 644
   Image URLs: ['https://images-na.ssl-images-amazon.com/images/I/61DvLcapd8L._SY88.jpg']
   Review: I went from fe